In [0]:
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from xgboost import plot_importance
import numpy as np
from sklearn.metrics import confusion_matrix


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [0]:
from google.colab import files
#uploaded = files.upload()
uploaded = files.upload()



Saving train.csv to train (4).csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [0]:
df.set_index('EmployeeNo',inplace=True)

In [0]:
uploaded2 = files.upload()

Saving test.csv to test (1).csv


In [0]:
#uploaded2 = files.upload()

In [0]:
df2 = pd.read_csv(io.BytesIO(uploaded2['test.csv']))

In [0]:
df2.set_index('EmployeeNo', inplace= True)
df2['Promoted_or_Not'] = 5
df2.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
EmployeeNo,,,,,,,,,,,,,,,,,,
YAK/S/00005,Information Technology and Solution Support,First Degree or HND,Male,Agency and others,2,1976,7.5,2017,0,0,65,FCT,Yes,Married,No,No,1,5
YAK/S/00011,Information Technology and Solution Support,NaN,Male,Direct Internal process,2,1991,0.0,2018,0,0,69,OGUN,Yes,Married,No,No,1,5
YAK/S/00015,Research and Innovation,"MSc, MBA and PhD",Male,Direct Internal process,2,1984,7.5,2012,0,0,76,KANO,Yes,Married,No,No,1,5
YAK/S/00016,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1984,2.5,2009,0,0,52,RIVERS,No,Single,No,No,1,5
YAK/S/00017,Information Technology and Solution Support,First Degree or HND,Male,Agency and others,2,1983,7.5,2014,1,0,69,FCT,Yes,Married,No,No,0,5


In [0]:
dfmerged = pd.concat([df,df2])
print(dfmerged.shape)
print(dfmerged.columns)

(54808, 18)
Index(['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment',
       'Trainings_Attended', 'Year_of_birth', 'Last_performance_score',
       'Year_of_recruitment', 'Targets_met', 'Previous_Award',
       'Training_score_average', 'State_Of_Origin', 'Foreign_schooled',
       'Marital_Status', 'Past_Disciplinary_Action',
       'Previous_IntraDepartmental_Movement', 'No_of_previous_employers',
       'Promoted_or_Not'],
      dtype='object')


In [0]:
#df['Last_performance_score'].describe()

In [0]:
df['Qualification'].describe()

count                   36633
unique                      3
top       First Degree or HND
freq                    25578
Name: Qualification, dtype: object

In [0]:
dfmerged['Qualification'].fillna('First Degree or HND', inplace=True)

In [0]:
#dividing year of birth by group
dfmerged['Year_of_birth'] = dfmerged['Year_of_birth'].astype(int)
dfmerged.loc[dfmerged['Year_of_birth'] <= 1950, 'Year_of_birth'] = 0
dfmerged.loc[(dfmerged['Year_of_birth'] > 1950) & (dfmerged['Year_of_birth'] <= 1955), 'Year_of_birth'] = 1
dfmerged.loc[(dfmerged['Year_of_birth'] > 1955) & (dfmerged['Year_of_birth'] <= 1960), 'Year_of_birth'] = 2
dfmerged.loc[(dfmerged['Year_of_birth'] > 1960) & (dfmerged['Year_of_birth'] <= 1965), 'Year_of_birth'] = 3
dfmerged.loc[(dfmerged['Year_of_birth'] > 1965) & (dfmerged['Year_of_birth'] <= 1970), 'Year_of_birth'] = 4
dfmerged.loc[(dfmerged['Year_of_birth'] > 1970) & (dfmerged['Year_of_birth'] <= 1975), 'Year_of_birth'] = 5
dfmerged.loc[(dfmerged['Year_of_birth'] > 1975) & (dfmerged['Year_of_birth'] <= 1980), 'Year_of_birth'] = 6
dfmerged.loc[(dfmerged['Year_of_birth'] > 1980) & (dfmerged['Year_of_birth'] <= 1985), 'Year_of_birth'] = 7
dfmerged.loc[(dfmerged['Year_of_birth'] > 1985) & (dfmerged['Year_of_birth'] <= 1990), 'Year_of_birth'] = 8
dfmerged.loc[(dfmerged['Year_of_birth'] > 1990) & (dfmerged['Year_of_birth'] <= 1995), 'Year_of_birth'] = 9
dfmerged.loc[(dfmerged['Year_of_birth'] > 1995) & (dfmerged['Year_of_birth'] <= 2000), 'Year_of_birth'] = 10
dfmerged.loc[ dfmerged['Year_of_birth'] > 2000, 'Year_of_birth'] = 11

#turning number of previous employer 'more than 5' into integer 6
dfmerged.loc[dfmerged['No_of_previous_employers'] == 'More than 5', 'No_of_previous_employers'] = 6
#converting the previous employer column from string into int
dfmerged['No_of_previous_employers'] = dfmerged['No_of_previous_employers'].astype(int)

#dividing year of recruitment by group
dfmerged['Year_of_recruitment'] = dfmerged['Year_of_recruitment'].astype(int)
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 1982) & (dfmerged['Year_of_recruitment'] <= 1985), 'Year_of_recruitment'] = 0
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 1985) & (dfmerged['Year_of_recruitment'] <= 1988), 'Year_of_recruitment'] = 1
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 1988) & (dfmerged['Year_of_recruitment'] <= 1991), 'Year_of_recruitment'] = 2
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 1991) & (dfmerged['Year_of_recruitment'] <= 1994), 'Year_of_recruitment'] = 3
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 1994) & (dfmerged['Year_of_recruitment'] <= 1997), 'Year_of_recruitment'] = 4
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 1997) & (dfmerged['Year_of_recruitment'] <= 2000), 'Year_of_recruitment'] = 5
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 2000) & (dfmerged['Year_of_recruitment'] <= 2003), 'Year_of_recruitment'] = 6
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 2003) & (dfmerged['Year_of_recruitment'] <= 2006), 'Year_of_recruitment'] = 7
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 2006) & (dfmerged['Year_of_recruitment'] <= 2009), 'Year_of_recruitment'] = 8
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 2009) & (dfmerged['Year_of_recruitment'] <= 2012), 'Year_of_recruitment'] = 9
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 2012) & (dfmerged['Year_of_recruitment'] <= 2015), 'Year_of_recruitment'] = 10
dfmerged.loc[(dfmerged['Year_of_recruitment'] > 2015) & (dfmerged['Year_of_recruitment'] <= 2018), 'Year_of_recruitment'] = 11
dfmerged.loc[ dfmerged['Year_of_recruitment'] > 2018, 'Year_of_recruitment'] = 12

In [0]:
#one hot encoding
dfmerged = dfmerged.drop(['State_Of_Origin'], axis=1)
dfmerged = pd.get_dummies(dfmerged, drop_first = True)
#print(training_data.iloc[0:5,:])
print(dfmerged.columns)
print(dfmerged.shape)

Index(['Trainings_Attended', 'Year_of_birth', 'Last_performance_score',
       'Year_of_recruitment', 'Targets_met', 'Previous_Award',
       'Training_score_average', 'No_of_previous_employers', 'Promoted_or_Not',
       'Division_Commercial Sales and Marketing',
       'Division_Customer Support and Field Operations',
       'Division_Information Technology and Solution Support',
       'Division_Information and Strategy', 'Division_People/HR Management',
       'Division_Regulatory and Legal services',
       'Division_Research and Innovation', 'Division_Sourcing and Purchasing',
       'Qualification_MSc, MBA and PhD',
       'Qualification_Non-University Education', 'Gender_Male',
       'Channel_of_Recruitment_Direct Internal process',
       'Channel_of_Recruitment_Referral and Special candidates',
       'Foreign_schooled_Yes', 'Marital_Status_Not_Sure',
       'Marital_Status_Single', 'Past_Disciplinary_Action_Yes',
       'Previous_IntraDepartmental_Movement_Yes'],
      dtyp

In [0]:
#dfmerged = pd.concat([dfmerged[['Trainings_Attended','Year_of_birth','Last_performance_score','Year_of_recruitment','Targets_met','Previous_Award','Training_score_average','No_of_previous_employers','Promoted_or_Not']],
#               pd.get_dummies(dfmerged['Division'],drop_first = True),pd.get_dummies(dfmerged['Qualification'],drop_first = True),pd.get_dummies(dfmerged['Gender'],drop_first = True),pd.get_dummies(dfmerged['Channel_of_Recruitment'],drop_first = True),pd.get_dummies(dfmerged['Foreign_schooled'],drop_first = True),pd.get_dummies(dfmerged['Marital_Status'],drop_first = True),pd.get_dummies(dfmerged['Past_Disciplinary_Action'],drop_first = True),pd.get_dummies(dfmerged['Previous_IntraDepartmental_Movement'],drop_first = True)],axis=1)

In [0]:
###Generating all possible pair of interactions between 2 pair of columns.
##Then removing those columns 

from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

Y = dfmerged.Promoted_or_Not
X = dfmerged.drop(['Promoted_or_Not'],1)


In [0]:
def add_interactions(df):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns)+['_'.join(x) for x in combos]
    scaler = MinMaxScaler()
    scaler.fit(df)
    df = scaler.transform(df)
    #scaler = MinMaxScaler()
    #scaler.fit(df)
    #df = scaler.transform(df)
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indices = [i for i,x in enumerate(list((df==0).all())) if x]
    df= df.drop(df.columns[noint_indices], axis=1)
    
    return df
X = add_interactions(X)
X[-1:-5]

,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,No_of_previous_employers,Division_Commercial Sales and Marketing,Division_Customer Support and Field Operations,Division_Information Technology and Solution Support,Division_Information and Strategy,Division_People/HR Management,Division_Regulatory and Legal services,Division_Research and Innovation,Division_Sourcing and Purchasing,"Qualification_MSc, MBA and PhD",Qualification_Non-University Education,Gender_Male,Channel_of_Recruitment_Direct Internal process,Channel_of_Recruitment_Referral and Special candidates,Foreign_schooled_Yes,Marital_Status_Not_Sure,Marital_Status_Single,Past_Disciplinary_Action_Yes,Previous_IntraDepartmental_Movement_Yes,Trainings_Attended_Year_of_birth,Trainings_Attended_Last_performance_score,Trainings_Attended_Year_of_recruitment,Trainings_Attended_Targets_met,Trainings_Attended_Previous_Award,Trainings_Attended_Training_score_average,Trainings_Attended_No_of_previous_employers,Trainings_Attended_Division_Commercial Sales and Marketing,Trainings_Attended_Division_Customer Support and Field Operations,Trainings_Attended_Division_Information Technology and Solution Support,Trainings_Attended_Division_Information and Strategy,Trainings_Attended_Division_People/HR Management,Trainings_Attended_Division_Regulatory and Legal services,Trainings_Attended_Division_Research and Innovation,Trainings_Attended_Division_Sourcing and Purchasing,"Trainings_Attended_Qualification_MSc, MBA and PhD",Trainings_Attended_Qualification_Non-University Education,Trainings_Attended_Gender_Male,Trainings_Attended_Channel_of_Recruitment_Direct Internal process,Trainings_Attended_Channel_of_Recruitment_Referral and Special candidates,Trainings_Attended_Foreign_schooled_Yes,Trainings_Attended_Marital_Status_Not_Sure,Trainings_Attended_Marital_Status_Single,Trainings_Attended_Past_Disciplinary_Action_Yes,Trainings_Attended_Previous_IntraDepartmental_Movement_Yes,Year_of_birth_Last_performance_score,Year_of_birth_Year_of_recruitment,Year_of_birth_Targets_met,Year_of_birth_Previous_Award,Year_of_birth_Training_score_average,Year_of_birth_No_of_previous_employers,Year_of_birth_Division_Commercial Sales and Marketing,Year_of_birth_Division_Customer Support and Field Operations,Year_of_birth_Division_Information Technology and Solution Support,Year_of_birth_Division_Information and Strategy,Year_of_birth_Division_People/HR Management,Year_of_birth_Division_Regulatory and Legal services,Year_of_birth_Division_Research and Innovation,Year_of_birth_Division_Sourcing and Purchasing,"Year_of_birth_Qualification_MSc, MBA and PhD",Year_of_birth_Qualification_Non-University Education,Year_of_birth_Gender_Male,Year_of_birth_Channel_of_Recruitment_Direct Internal process,Year_of_birth_Channel_of_Recruitment_Referral and Special candidates,Year_of_birth_Foreign_schooled_Yes,Year_of_birth_Marital_Status_Not_Sure,Year_of_birth_Marital_Status_Single,Year_of_birth_Past_Disciplinary_Action_Yes,Year_of_birth_Previous_IntraDepartmental_Movement_Yes,Last_performance_score_Year_of_recruitment,Last_performance_score_Targets_met,Last_performance_score_Previous_Award,Last_performance_score_Training_score_average,Last_performance_score_No_of_previous_employers,Last_performance_score_Division_Commercial Sales and Marketing,Last_performance_score_Division_Customer Support and Field Operations,Last_performance_score_Division_Information Technology and Solution Support,Last_performance_score_Division_Information and Strategy,Last_performance_score_Division_People/HR Management,Last_performance_score_Division_Regulatory and Legal services,Last_performance_score_Division_Research and Innovation,Last_performance_score_Division_Sourcing and Purchasing,"Last_performance_score_Qualification_MSc, MBA and PhD",Last_performance_score_Qualification_Non-University Education,Last_performance_score_Gender_Male,Last_performance_score_Channel_of_Recr

In [0]:
#X_check = X.loc[:,X.columns.duplicated()]
#X_check

In [0]:
X.shape

(54808, 317)

In [0]:
###Seperate out the training and testing dataset
X.index = Y.index
X['Promoted_or_Not'] = Y

X_validation = X[38312:]
X = X[0:38312]
X_validation.shape

(16496, 318)

In [0]:
Y = X.Promoted_or_Not
X = X.drop(['Promoted_or_Not'],1)

In [0]:
print(Y.shape)
print(X.shape)

(38312,)
(38312, 317)


In [0]:
seed = 2
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [0]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(30649, 317)
(30649,)
(7663, 317)
(7663,)


In [0]:
###Hypertune the model intensively with a 5 fold cross validation strategy.
##first grid search for max_depth,min_child_weight then fix those and search for rest in same fashion
### parameter scale_pos_weight is quite important in case of imbalanced dataset

from sklearn.model_selection import GridSearchCV
param_test2b = {
  #'min_child_weight' : [5,6],
   # 'max_depth':range(3,10,2),
  #'n_estimators':[150,200,300,400],
   # 'n_estimators':[150,200,300,400],
    'scale_pos_weight':[1,2,3,4],
   # 'colsample':[0.7,0.8]
    'colsample_bytree':[0.7,0.8],
    'subsample':[0.7,0.8],
    'gamma':[0,0.2,0.4]
   # 'min_child_weight' : [5,6],
    #'max_depth':range(3,10,2),
    #'min_child_weight':[5,6],
  #'max_depth': range(3,10,2),
  #'n_estimators':[150,200,300,400],
  #'scale_pos_weight':[1,2,3,4],
  #  'colsample_bytree':[0.7,0.8], 
  #'subsample':[0.7,0.8],
  #'gamma':[0,0.2,0.4]
    
}

In [0]:
#from sklearn.grid_search import GridSearchCV   #Perforing grid search

gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=150, max_depth=3,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=3,seed=14), 
 param_grid = param_test2b, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch2b.fit(X_train, y_train)
#print(gsearch2b.grid_scores_)
print("gsearch2b.best_params_",gsearch2b.best_params_)
print("gsearch2b.best_score_",gsearch2b.best_score_)



In [0]:
###Train the model with the best params
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=150, max_depth=3, min_child_weight=6, 
                      gamma=0, nthread=4, subsample=0.7, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=2,seed=14)
modelXg.fit(X_train, y_train)
y_xg = modelXg.predict(X_test)

In [0]:
## Determine whether your model is overfitting or not , with the help of ROC.
print(confusion_matrix(y_test, y_xg))
predictions = [value for value in y_xg]
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))
print("precision_score: %.2f%% on test dataset" % (precision * 100.0))
print("recall_score: %.2f%% on test dataset" % (recall * 100.0))
print("f1_score: %.2f%% on test dataset" % (f1 * 100.0))
print("roc_auc test set", roc_auc_score(y_test, modelXg.predict_proba(X_test)[:,1]))
print("roc_auc training set", roc_auc_score(y_train, modelXg.predict_proba(X_train)[:,1]))

[[6958   53]
 [ 417  235]]
Accuracy_score: 93.87% on test dataset
precision_score: 81.60% on test dataset
recall_score: 36.04% on test dataset
f1_score: 50.00% on test dataset
roc_auc test set 0.8962481613030531
roc_auc training set 0.9188203862660353


In [0]:
##Once the model is ready, train the model on entire dataset.
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=150, max_depth=3, min_child_weight=6, 
                      gamma=0,nthread=4, subsample=0.7, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=2,seed=14)

modelXg.fit(X, Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=6, missing=None, n_estimators=150, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=2, seed=14,
              silent=None, subsample=0.7, verbosity=1)

In [0]:
'''
##find the useful cols
usefulcols = []
for i in sorted( model.booster().get_fscore().items(), key=lambda x: x[1], reverse=True)[:100]:
    usefulcols.append(i[0])
print usefulcols
'''

In [0]:
X = X_validation.drop(['Promoted_or_Not'],1)
dfvalidation = X_validation.drop(['Promoted_or_Not'],1)
#y_valid = modelXg.predict(dfvalidation)
#submission = dfvalidation.copy()
#submission['Promoted_or_Not'] = y_valid
#submission['Promoted_or_Not'].value_counts()

In [0]:
df2 = pd.read_csv(io.BytesIO(uploaded2['test.csv']))
df2validation_id = df2['EmployeeNo']

In [0]:
X = X_validation.drop(['Promoted_or_Not'],1)
dfvalidation = X_validation.drop(['Promoted_or_Not'],1)
final_pred = modelXg.predict(dfvalidation)
final_promotion_pred = pd.DataFrame(final_pred, columns=['Promoted_or_Not'], index=df2validation_id)
print(final_promotion_pred)

             Promoted_or_Not
EmployeeNo                  
YAK/S/00005                0
YAK/S/00011                0
YAK/S/00015                0
YAK/S/00016                0
YAK/S/00017                0
YAK/S/00019                0
YAK/S/00027                0
YAK/S/00028                0
YAK/S/00031                0
YAK/S/00032                0
YAK/S/00034                0
YAK/S/00038                0
YAK/S/00040                0
YAK/S/00041                0
YAK/S/00043                0
YAK/S/00045                0
YAK/S/00047                0
YAK/S/00049                0
YAK/S/00050                0
YAK/S/00051                0
YAK/S/00058                0
YAK/S/00060                0
YAK/S/00063                0
YAK/S/00067                0
YAK/S/00069                0
YAK/S/00072                0
YAK/S/00080                1
YAK/S/00083                0
YAK/S/00084                0
YAK/S/00088                0
YAK/S/00092                0
YAK/S/00096                0
YAK/S/00099   

In [0]:
#index = df2['EmployeeNo']
#submission = submission[['Promoted_or_Not']]
#submission = submission[['is_promoted']]
#submission.reset_index(inplace=True)

from google.colab import files
final_promotion_pred.to_csv('employee_promotion_6_xgb.csv')
#submission.to_csv('employee_promotion_6_xgb.csv',index=index) 
files.download('employee_promotion_6_xgb.csv')


#submission.to_csv("Promoted or not submission 6", index=index)
#submission.to_csv("Promoted or not submission 6",index=False)
#print(submission)

In [0]:
'''sub = pd.read_csv("solution_18sept.csv")

private = pd.read_csv("private_W1BajhK.csv")
private.columns = ['employee_id', 'actual']
private = private.merge(sub, on = "employee_id", how="left")'''

In [0]:
'''from sklearn.metrics import f1_score

f1_score(private['actual'], private['is_promoted'])'''